In [5]:
import pandas as pd
import numpy as np

In [6]:
dff = pd.read_csv("data/Main_Demo.csv")

In [7]:
dff['Bill'] = dff['Bill'].astype(np.int64)

In [8]:
dff['BillDate'] = pd.to_datetime(dff['BillDate'])

In [9]:
dff['CustomerID'] = dff['CustomerID'].astype(int)

In [10]:
Last_Date = dff['BillDate'].max()

In [11]:
print(Last_Date)

2019-12-10 00:00:00


In [13]:
RFM_table = dff.groupby(by = ['CustomerID'], as_index = False).agg({'BillDate':lambda x: (Last_Date- x.max()).days,'Bill': lambda x: len(x), 'TotalAmount':lambda x: x.sum()})

In [14]:
RFM_table.rename(columns = {'BillDate':'Recency', 'Bill':'Frequency', 'TotalAmount':'Monetary'}, inplace = True)


In [15]:
RFM_table

,CustomerID,Recency,Frequency,Monetary
0,12354,233,1,16.95
1,12355,568,1,78.00
2,12357,389,1,218.40
3,12358,850,1,417.00
4,12360,53,1,37.90
...,...,...,...,...
1188,18260,173,1,16.50
1189,18261,215,1,23.00
1190,18276,599,1,34.20
1191,18283,60,1,8.98


In [16]:
RFM_table["Recency_Score"] = pd.qcut(RFM_table['Recency'], 5, labels=[5, 4, 3, 2, 1])

In [17]:
RFM_table

,CustomerID,Recency,Frequency,Monetary,Recency_Score
0,12354,233,1,16.95,3
1,12355,568,1,78.00,1
2,12357,389,1,218.40,3
3,12358,850,1,417.00,1
4,12360,53,1,37.90,5
...,...,...,...,...,...
1188,18260,173,1,16.50,4
1189,18261,215,1,23.00,3
1190,18276,599,1,34.20,1
1191,18283,60,1,8.98,5


In [18]:
RFM_table["Frequency_Score"] = pd.qcut(RFM_table["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [19]:
RFM_table

,CustomerID,Recency,Frequency,Monetary,Recency_Score,Frequency_Score
0,12354,233,1,16.95,3,1
1,12355,568,1,78.00,1,1
2,12357,389,1,218.40,3,1
3,12358,850,1,417.00,1,1
4,12360,53,1,37.90,5,1
...,...,...,...,...,...,...
1188,18260,173,1,16.50,4,4
1189,18261,215,1,23.00,3,4
1190,18276,599,1,34.20,1,4
1191,18283,60,1,8.98,5,4


In [20]:
RFM_table["Monetary_Score"] = pd.qcut(RFM_table["Monetary"], 5, labels=[1, 2, 3, 4, 5])

In [22]:
RFM_table

,CustomerID,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score
0,12354,233,1,16.95,3,1,2
1,12355,568,1,78.00,1,1,4
2,12357,389,1,218.40,3,1,5
3,12358,850,1,417.00,1,1,5
4,12360,53,1,37.90,5,1,3
...,...,...,...,...,...,...,...
1188,18260,173,1,16.50,4,4,2
1189,18261,215,1,23.00,3,4,2
1190,18276,599,1,34.20,1,4,3
1191,18283,60,1,8.98,5,4,1


In [23]:
RFM_table["RFM_Score"] = (RFM_table["Recency_Score"].astype(str) + RFM_table["Frequency_Score"].astype(str))

In [24]:
RFM_table

,CustomerID,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RFM_Score
0,12354,233,1,16.95,3,1,2,31
1,12355,568,1,78.00,1,1,4,11
2,12357,389,1,218.40,3,1,5,31
3,12358,850,1,417.00,1,1,5,11
4,12360,53,1,37.90,5,1,3,51
...,...,...,...,...,...,...,...,...
1188,18260,173,1,16.50,4,4,2,44
1189,18261,215,1,23.00,3,4,2,34
1190,18276,599,1,34.20,1,4,3,14
1191,18283,60,1,8.98,5,4,1,54


In [26]:
seg_map = {r'[1-2][1-2]': 'Hibernating',r'[1-2][3-4]': 'At_Risk',r'[1-2]5': 'Cant_loose',r'3[1-2]': 'About_to_sleep',r'33': 'Need_attention',r'[3-4][4-5]': 'Loyal_customers',r'41': 'Promising',r'51': 'New_customers',r'[4-5][2-3]': 'Potential_loyalists',r'5[4-5]': 'Champions'}
RFM_table['Segment'] = RFM_table['RFM_Score'].replace(seg_map, regex=True)   

In [27]:
RFM_table

,CustomerID,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RFM_Score,Segment
0,12354,233,1,16.95,3,1,2,31,About_to_sleep
1,12355,568,1,78.00,1,1,4,11,Hibernating
2,12357,389,1,218.40,3,1,5,31,About_to_sleep
3,12358,850,1,417.00,1,1,5,11,Hibernating
4,12360,53,1,37.90,5,1,3,51,New_customers
...,...,...,...,...,...,...,...,...,...
1188,18260,173,1,16.50,4,4,2,44,Loyal_customers
1189,18261,215,1,23.00,3,4,2,34,Loyal_customers
1190,18276,599,1,34.20,1,4,3,14,At_Risk
1191,18283,60,1,8.98,5,4,1,54,Champions
